# Fine-tuning a model on a translation task

In [44]:
import transformers
model_checkpoint = "Helsinki-NLP/opus-mt-de-en"

This pretrained model comes from [Model Hub](https://huggingface.co/models)

## Loading the dataset

In [45]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("wmt16", "de-en")
metric = load_metric("sacrebleu")

Reusing dataset wmt16 (/home/dbr/.cache/huggingface/datasets/wmt16/de-en/1.0.0/9e0038fe4cc117bd474d2774032cc133e355146ed0a47021b2040ca9db4645c0)


  0%|          | 0/3 [00:00<?, ?it/s]

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set:

In [22]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 4548885
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2169
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2999
    })
})

In [46]:
raw_datasets["train"][0]

{'translation': {'de': 'Wiederaufnahme der Sitzungsperiode',
  'en': 'Resumption of the session'}}

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [47]:
import datasets
import random
import pandas as pd

def show_random_elements(dataset, num_examples=5):
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [48]:
show_random_elements(raw_datasets["train"])

,translation
0,"{'de': 'Uns ist bewusst, dass diese Thesen sehr abstrakt gehalten sind – sie müssen es auch sein, da sie weniger auf konkreten eigenen Erfahrungen, als auf theoretischen und historischen Überlegungen fußen.', 'en': 'We are aware that these theses are very abstract – they must be, since they are based more on theoretical and historical considerations than on concrete experience (and they are focused on the trade unions in Germany in particular).'}"
1,"{'de': 'Unsere Zusagen müssen zu konkreten Aktionen führen.', 'en': 'Our commitments need to be followed by specific actions.'}"
2,"{'de': 'Durch identische Befestigungspunkte lassen sich aber beide Arten durch die neue Pickupplatine ersetzen.', 'en': 'Because of identical fastening spots however both types can be replaced by the new Pickup-PCB.'}"
3,"{'de': 'Ein generelles Verbot bestimmter Forschungsrichtungen birgt eine erhöhte Gefahr für Missbrauch und Spekulationen.', 'en': 'A total ban on certain forms of research leads to greater risks of abuse and risk-taking.'}"
4,"{'de': 'Nationales Weinbauzentrum arbeitet bei der Unterstützung und bei der Werbung eng mit anderen Weinbauorganisationen, staatlichen und regionalen Organen in der Tschechischen Republik zusammen.', 'en': 'The National Wine Centre during promotion and support closely cooperates with other wine-growing organisations, governmental and regional authorities in the Czech Republic.'}"


In [8]:
metric

Metric(name: "sacrebleu", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, usage: """
Produces BLEU scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: The system stream (a sequence of segments).
    references: A list of one or more reference streams (each a sequence of segments).
    smooth_method: The smoothing method to use. (Default: 'exp').
    smooth_value: The smoothing value. Only valid for 'floor' and 'add-k'. (Defaults: floor: 0.1, add-k: 1).
    tokenize: Tokenization method to use for BLEU. If not provided, defaults to 'zh' for Chinese, 'ja-mecab' for
        Japanese and '13a' (mteval) otherwise.
    lowercase: Lowercase the data. If True, enables case-insensitivity. (Default: False).
    force: Insist that your tokenized input is actually detokenized.

Returns:
    'score': BLEU score,
    'counts'

You can call its `compute` method with your predictions and labels, which need to be list of decoded strings (list of list for the labels):

In [49]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = [["hello there"], ["general kenobi"]]

metric.compute(predictions=fake_preds, references=fake_labels)

{'score': 0.0,
 'counts': [4, 2, 0, 0],
 'totals': [4, 2, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'bp': 1.0,
 'sys_len': 4,
 'ref_len': 4}

## Preprocessing the data

In [50]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [38]:
tokenizer("Hello, this one sentence!")

{'input_ids': [8152, 166, 2, 60, 32, 18, 7795, 46, 2066, 68, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [13]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[8152, 166, 2, 60, 32, 18, 7795, 46, 2066, 68, 0], [140, 19, 39, 5748, 45, 7795, 46, 2066, 3, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

To prepare the targets for our model, we need to tokenize them inside the `as_target_tokenizer` context manager. This will make sure the tokenizer uses the special tokens corresponding to the targets:

In [14]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[16816, 2, 60, 128, 12512, 68, 0], [140, 19, 811, 12512, 3, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


In [51]:
prefix = ""

We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model. The padding will be dealt with later on (in a data collator) so we pad examples to the longest length in the batch and not the whole dataset.

In [41]:
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "de"

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [54]:
from pprint import pprint
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[6053, 477, 12353, 7, 4, 2952, 6, 2314, 0], [38, 230, 6359, 4006, 371, 8403, 108, 4, 2952, 6, 2314, 7, 4, 151, 3336, 1043, 7096, 3059, 3075, 705, 34, 108, 32, 7135, 3136, 507, 9851, 418, 935, 1958, 2, 8, 38, 563, 10382, 6843, 920, 32, 1214, 17, 11514, 118, 12, 3066, 1870, 41, 14, 3860, 20133, 168, 17, 3722, 5, 4, 6516, 1543, 35, 41, 17, 15, 3075, 132, 108, 14, 17, 5178, 817, 1085, 1772, 1013, 343, 1544, 82, 3, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[1699, 10885, 1726, 9106, 1032, 9, 124, 4935, 1509, 37820, 18, 0], [38, 492, 322, 7072, 2836, 135, 11, 121, 4407, 2706, 2, 57, 507, 3, 599, 239, 31534, 318, 621, 5989, 1017, 18, 124, 4935, 1509, 37820, 18, 40, 707, 503, 1258, 801, 3336, 1270, 5109, 28, 21206, 10885, 1726, 749, 3

In [43]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/4549 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

## Fine-tuning the model

In [19]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/284M [00:00<?, ?B/s]

In [57]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]

args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True
)

In [58]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [60]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [61]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [62]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: translation.
/home/dbr/miniconda3/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 4548885
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 284306


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 